In [ ]:
import os
import robotic as ry
import numpy as np
import matplotlib.pyplot as plt
import random
import time
import torch

C = ry.Config()
#C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))
robot="../rai-robotModels/panda/panda_tunnel.g"
C.addFile(robot)
C.addFile("shelf_env.g")
C.addFile("extras.g")

C.addFile("camera.g")
C.view()
config=C

In [ ]:
#Initialize Action/Path Tree
action_tree=[]

#Collect Frame States (it goes incrementally according to t. First frame is initial Config)
frame_list=[]

#Goal_Pose and Contact_Point list
final_pose_list= []
contact_point_list= []

#Action Hyperparameters
delta=0.2 #Duration of the action (Such as distance between nodes)
phase=30 #Komo stepsPerPhase
trial=8 #Number of action trials
max_iter=40 #Max iteration number of main loop (=WP count) (=len(path))
rrt_extend_length=0.1 #For RRT (not used right now)
threshold= 0.08 # Threshold of goal pose

book = config.frame("book")
obs = config.frame("book_obstacle")
toole = config.frame("stick")

path_list = [[config.getJointState()]]
object_list = [[*book.getPosition(), *book.getQuaternion()]]
obs_list =  [[*obs.getPosition(), *obs.getQuaternion()]]
tool_list = [[*toole.getPosition(), *toole.getQuaternion()]]

In [ ]:
def RRT(qF, rrt_extend_length=0.04):
    tic = time.time()
    C2 = ry.Config()
    C2.addFile("shelf_actuated_env.g")
    
    q0 = C2.getJointState()
    print("Start State: ", q0," Final State: ", qF)
    C2.view()

    # Create an RRT path finder
    rrt = ry.PathFinder()
    rrt.setProblem(C2, [q0], [qF])
    
    isFeas = False
    path = None

    i=0
    # Find an RRT path
    while i!=10 and (not isFeas):
        rrt = ry.PathFinder()
        rrt.setProblem(C2, [q0], [qF])
        ret = rrt.solve()
        isFeas = ret.feasible
        path = ret.x
        i+=1
    

    ## Smooth the path (optimization) ##
    print("Path Prev:", len(path))
    #Create a KOMO for post-optimization with acceleration constraints
    num_time_steps = len(path)

    komo2 = ry.KOMO(C2, num_time_steps, 1, 2, True)
    #Add an acceleration constraint objective to the new KOMO object
    komo2.addControlObjective([], 2, 1e0)
    komo2.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e2])
    #Set the waypoints for the KOMO
    komo2.initWithPath_qOrg(path)

    ret = ry.NLP_Solver() \
        .setProblem(komo2.nlp()) \
        .setOptions(stopTolerance=1e-2, verbose=0) \
        .solve()
    print(ret)

    path = komo2.getPath()
    print("RRT Len:", len(path))

    new_path = path
    toc = time.time()
    print(f"RRT time: {toc-tic}")

    for i in range(0, len(new_path)):
        C2.setJointState(new_path[i])
        C2.view()
        time.sleep(.1)

    return new_path

In [ ]:
qF = [0.14, 0.58, 1.24, -1, 0, 0, 0]
path = RRT(qF)
goal_pos=qF

In [ ]:
#### Use Vision/ Point cloud for CP estimation ####
def getCameraView(C, camName, filter = 10, plot = True):
    cam = ry.CameraView(C)
    cam.setCamera(camName)
    rgb, depth = cam.computeImageAndDepth(C)

    # Filtering the object
    mask = np.zeros(rgb[:,:,1].shape)
    red = rgb[:,:,0]
    green = rgb[:,:,1]
    blue = rgb[:,:,2]
    
    (x, y) = mask.shape
    for i in range(0, x):
        for j in range(0, y):
            if green[i][j] > 180 and blue[i][j] > 180 and red[i][j] < 50:
                mask[i][j] = 1
                
    newDepth = np.where(mask, depth, np.nan)
    pcl = ry.depthImage2PointCloud(newDepth, cam.getFxycxy())

    # Removing the other points
    cloud = []
    (x, y) = depth.shape
    for i in range(0, x):
        for j in range(0, y):
            if mask[i][j] and j%filter == 0:
                cloud.append(pcl[i][j])
    cloud = np.asarray(cloud)
    
    if cloud.size!=0:
        
        if plot:
            fig = plt.figure()
            fig.add_subplot(1,2,1)
            plt.imshow(rgb)
            fig.add_subplot(1,2,2)
            plt.imshow(depth)
            plt.show()

    return [cam, cloud]

prev_c=[]
def findGraspPoint(C, c):
    global prev_c
    if c.size==0:
        print("Couldn't find point cloud")
        c=prev_c
        
    cam = C.frame("cam_5")
    grasp_point = cam.getPosition() - np.array(c[random.randint(0, (c.shape[0] - 1)), :])
    grasp_point[0] *= -1
    prev_c = c
    return grasp_point

def isContactFeasible(C, t, contact_point):
    S = ry.Skeleton()
    S.enableAccumulatedCollisions(True)
    S.addEntry([t, t], ry.SY.touch,  ["gripper", contact_point])
    komo = S.getKomo_path(C, stepsPerPhase=30, accScale=1e0, lenScale=1e-2, homingScale=1e-1, collScale=1e3)
    ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
    print(ret)
    return ret.feasible

def addMarker(C, pos, name, quat = [1, 0, 0, 0], obj= "book"):
    marker = C.addFrame(name, parent= obj) \
              .setPosition([*pos[:2], pos[2]]) \
              .setQuaternion(quat)
    return marker

def convertCPrelative(C,new_name="rel_contact_point",name="contact_point", obj= "book"):
    relativeCP= config.getFrame(name)
    rel_CP_pos=relativeCP.getRelativePosition()
    relCP = C.addFrame(new_name, parent= obj) \
              .setRelativePosition(rel_CP_pos) 

#Generate Contact point
def CPgeneration(C, obj= "book"):
    [p1, c] = getCameraView(C, "cam_1", 10, False)
    
    if c.size==0:
        return None
    grasp_point=findGraspPoint(C, c)
    print(f"Contact Point= {grasp_point}")
    marker = addMarker(C, grasp_point, "contact_point",obj=obj)
    convertCPrelative(C,"rel_contact_point", obj=obj)
    #marker = addMarker(C, grasp_point, "contact_point",  [0.7038453156522361, 0.0, 0.0, 0.7103532724176078])
    itr=0
    while itr!=10 and (not isContactFeasible(C, 1, "rel_contact_point")):
        itr+=1
        C.delFrame("contact_point")
        grasp_point = findGraspPoint(C, c)
        marker = addMarker(C, grasp_point, "contact_point",obj=obj)
        convertCPrelative(C,"rel_contact_point","contact_point",obj=obj)
        #marker = addMarker(C, grasp_point, "contact_point", 0.1, [0.7038453156522361, 0.0, 0.0, 0.7103532724176078])
        print(f"This Contact is feasible (found in {itr} iteration)")

    addPoint2(C, [0, -0.1, 0.], "grasp_conint", "book", [1, 1, 0])

#Pick Action
def pick(contact_point,final_pose,t=1):

    skeleton = ry.Skeleton()

    skeleton.addEntry([t, -1], ry.SY.stable,  ["gripper", "book"])
    skeleton.addEntry([t, -1], ry.SY.touch,  ["gripper","book"])
    skeleton.addEntry([t+delta, t+delta], ry.SY.poseEq, ["book", final_pose])
    #skeleton.enableAccumulatedCollisions(True)

    komo = skeleton.getKomo_path(config, stepsPerPhase=phase, accScale=1e0, lenScale=1e-2, homingScale=1e-2, collScale=1e1)
    #komo.addObjective([t, -1], ry.FS.positionDiff, ["gripper", contact_point], ry.OT.sos, target = [0.0], scale=[1e2])
    #komo.addObjective([t, -1], ry.FS.poseDiff, ["gripper", "book_cp"], ry.OT.sos, target = [0.0], scale=[1e2])
    komo.addObjective([t, -1], ry.FS.positionDiff, ["gripper", "book_cp"], ry.OT.sos, target = [0.0], scale=[1e2])
    ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
    print(ret)
    return komo,ret

#Pick Obs Action
def pick_obs(contact_point,final_pose,t=1):

    skeleton = ry.Skeleton()

    skeleton.addEntry([t, -1], ry.SY.stable,  ["gripper", "book_obstacle"])
    skeleton.addEntry([t, -1], ry.SY.touch,  ["gripper","book_obstacle"])
    skeleton.addEntry([t+delta, t+delta], ry.SY.poseEq, ["book_obstacle", final_pose])
    #skeleton.enableAccumulatedCollisions(True)

    komo = skeleton.getKomo_path(config, stepsPerPhase=phase, accScale=1e0, lenScale=1e-2, homingScale=1e-2, collScale=1e1)
    #komo.addObjective([t, -1], ry.FS.positionDiff, ["gripper", contact_point], ry.OT.sos, target = [0.0], scale=[1e2])
    ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
    print(ret)
    return komo,ret

#Push Actions
def push_stick_obs(contact_point,final_pose,t=1):

    skeleton = ry.Skeleton()
    skeleton.addEntry([t, -1], ry.SY.stable,  ["gripper", "stick"])
    skeleton.addEntry([t, -1], ry.SY.touch,  ["gripper", "stick"])  #Check this part for optimal motion

    skeleton.addEntry([t+delta, -1], ry.SY.stable,  ["stick","book_obstacle"])  
    skeleton.addEntry([t+delta, -1], ry.SY.touch,  ["stick","book_obstacle"])
    skeleton.addEntry([t+2*delta, t+2*delta], ry.SY.poseEq, ["book_obstacle", final_pose])
    skeleton.enableAccumulatedCollisions(True)

    komo = skeleton.getKomo_path(config, stepsPerPhase=phase, accScale=1e0, lenScale=1e-2, homingScale=1e-3, collScale=1e1)
    #komo.addObjective([t, -1], ry.FS.positionDiff, ["gripper", contact_point], ry.OT.sos, target = [0.0], scale=[1e2])
    ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
    print(ret)
    return komo,ret

#Push Actions
def push_stick(contact_point,final_pose,t=1):
    delta=0.4
    skeleton = ry.Skeleton()
    skeleton.addEntry([t, -1], ry.SY.stable,  ["gripper", "stick"])
    skeleton.addEntry([t, -1], ry.SY.touch,  ["gripper", "stick"])  #Check this part for optimal motion

    skeleton.addEntry([t+delta, -1], ry.SY.stable,  ["stick","book"])  
    skeleton.addEntry([t+delta, -1], ry.SY.touch,  ["stick","book_cp"])
    #skeleton.addEntry([t+delta, -1], ry.SY.touch,  ["stick","book"])
    skeleton.addEntry([t+delta+0.1, t+delta+0.1], ry.SY.poseEq, ["book", final_pose])
    skeleton.enableAccumulatedCollisions(True)

    komo = skeleton.getKomo_path(config, stepsPerPhase=120, accScale=1e0, lenScale=1e-2, homingScale=1e-3, collScale=1e1)
    komo.addObjective([t, -1], ry.FS.positionDiff, ["gripper", "stick_cp"], ry.OT.sos, target = [0.0], scale=[1e1])
    #komo.addObjective([t+delta, -1], ry.FS.positionDiff, ["stick","book_cp"], ry.OT.sos, target = [0.0], scale=[1e1])
    ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
    print(ret)
    return komo,ret

def place_stick(contact_point,final_pose,t=1):
    delta=0.5
    skeleton = ry.Skeleton()
    skeleton.addEntry([t, -1], ry.SY.stable,  ["gripper", "stick"])
    skeleton.addEntry([t, -1], ry.SY.touch,  ["gripper", "stick"])  #Check this part for optimal motion

    skeleton.addEntry([t+delta, t+2*delta], ry.SY.stable,  ["stick","book_obstacle"])  
    skeleton.addEntry([t+delta, t+2*delta], ry.SY.touch,  ["stick","book_obstacle"])
    skeleton.addEntry([t+2*delta, -1], ry.SY.poseEq, ["book_obstacle", final_pose])
    skeleton.addEntry([t+2*delta, -1], ry.SY.stable,  [final_pose,"book_obstacle"]) 
    skeleton.enableAccumulatedCollisions(True)
 
    skeleton.addEntry([t+3*delta, t+3*delta], ry.SY.poseEq, ["stick", "stick_initial"])
    #skeleton.enableAccumulatedCollisions(True)

    komo = skeleton.getKomo_path(config, stepsPerPhase=60, accScale=1e0, lenScale=1e-2, homingScale=1e-2, collScale=1e1)
    #komo.addObjective([t, -1], ry.FS.positionDiff, ["gripper", contact_point], ry.OT.sos, target = [0.0], scale=[1e2])
    ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
    print(ret)
    return komo,ret

#WP feasibility check
def tryWP(gen_wp,i):
    print(f"Generated WP is: {gen_wp}")

    #Create as frame
    wpx=config.addFrame(name=f'wp{i}')
    wpx.setPosition(gen_wp[0:3]) #Generate WP Position
    wpx.setQuaternion(gen_wp[3:])   
    addPoint2(config, [0, 0, 0], "point"+str(random.randint(0,10000)), "world", [1, 0, 0])
    return gen_wp    

def addPoint(C, pos, name, parent, color):
    point = C.addFrame(name, parent= parent) \
              .setShape(ry.ST.sphere, size = [0, 0, 0, .02]) \
              .setPosition(pos) \
              .setColor(color) \
              .setContact(0)
    C.view()
    return point

def addPoint2(C, pos, name, parent, color):
    pos = C.frame("book").getPosition()
    point = C.addFrame(name, parent= parent) \
              .setShape(ry.ST.sphere, size = [0, 0, 0, .02]) \
              .setRelativePosition(pos) \
              .setColor(color) \
              .setContact(0)
    C.view()
    return point

In [ ]:
def Point_Cloud_Flag(C):
    [p1, c] = getCameraView(C, "cam_1", 2, False)
    
    if c.size==0:
        return False
    else:
        return True

i=0
CPgeneration(config, obj= "book") #Initial CP generator, it uses path[0] (first WP)
#convertCPrelative(config,"contact_point", obj= "book")
gen_cp= "rel_contact_point" #This will come from CP creator
gen_wp=np.zeros(np.shape(goal_pos))
counter=0
use_tool=False
while i!=max_iter and not np.linalg.norm(gen_wp - goal_pos) <threshold:
    print(f'Step: {i+1}')
    fail=0
    flag=Point_Cloud_Flag(config)
    for n in range(trial):
        #gen_wp = WPgeneration() #Such as Extent function of RRT
        gen_wp=path[i]
        tryWP(gen_wp,i)
        #CPgeneration(config, obj= "stick")
        if n>=3 or use_tool==True:
            end_pose= f'wp{i}'
            if n>=3 and use_tool==False:
                t=1
                komo,ret = push_stick("aa",end_pose,t)
            
                #Visualize
                #komo.view(True, "waypoints solution")
                #komo.view_play(True, 0.5)
                #komo.report(False, plotOverTime=True)
                if ret.eq<=1.5:
                    use_tool=True
            elif use_tool==True:
                t=0
                komo,ret = push_stick("aa",end_pose,t)
            
                #Visualize
                #komo.view(True, "waypoints solution")
                #komo.view_play(True, 0.5)
                #komo.report(False, plotOverTime=True)
        else:
            if i!=0:    

                           
                #delta=0.4
                
                if n >=2:
                    CPgeneration(config, obj= "book") #This will become actual CP generator
                    convertCPrelative(config,"contact_point", obj= "book")
                    gen_cp= "rel_contact_point" 
                
                #Test Waypoint:#
                #Parameter Inıtialization
                end_pose= f'wp{i}'
                contact_point= gen_cp

                if i==1:    #There is a problem when t is not always equal to 1
                    CPgeneration(config, obj= "book") #This will become actual CP generator
                    convertCPrelative(config,"contact_point", obj= "book")
                    gen_cp= "rel_contact_point" 
                    contact_point= gen_cp                    
                    t=1
                    #Solve Pick Action with KOMO
                    komo,ret = pick(contact_point,end_pose,t)
                elif n>=2:   
                    t=0.4
                    #Solve push Action with KOMO
                    komo,ret = pick(contact_point,end_pose,t)
                else:
                    t=0
                    #Solve push Action with KOMO
                    komo,ret = pick(contact_point,end_pose,t)
                
                print(f"CONTACT POINT: {contact_point}")
            else:
                t=1
                #Solve Pick Action with KOMO
                #komo,ret = push_stick_obs("aa","obs_target",t)
                komo,ret = place_stick("aa","obs_target",t)


            #Visualize
            #komo.view(True, "waypoints solution")
            #komo.view_play(True, 0.5)
            #komo.report(False, plotOverTime=True)
        

        if ret.eq<=0.8:
            print("Action is feasible") 
            #Set Final Configuration of current action


           
            waypoints = komo.getPath_qAll()
            Frame_state=komo.getFrameState(len(waypoints)-1)
            frame_list.append(Frame_state)
            final_pose_list.append(gen_wp)
            contact_point_list.append(gen_cp)
            config.setFrameState(Frame_state)
            path_list.append(komo.getPath())
            object_list.append([*book.getPosition(), *book.getQuaternion()])

            tool_list.append([*toole.getPosition(), *toole.getQuaternion()])
            obs_list.append([*obs.getPosition(), *obs.getQuaternion()])
            config.view()
            break #Continue to next WP in the path

        else:
            print(f"Infeasible Action, Generate new WP,CP pair: {n}")
            feasibility= False
            fail+=1

    i+=1    
 
    if fail!=(trial):
        continue
    else:
        #Generate new WP list (path) in this case
        #WPgeneration()
        print("New path is calculated")
        print("Try to use smaller 'extend_length' or higher 'phase'. Play with Hyperparameters")
        print("Action Failed")
        break

if np.linalg.norm(gen_wp - goal_pos) <threshold:
    print("TRAJECTORY FOUND!!!!")

In [ ]:
### Visualization ###
environment="shelf_env_vis.g"
robot="../rai-robotModels/panda/panda_tunnel.g"
vis_config = ry.Config()

vis_config.addFile(robot)
vis_config.addFile(environment)
bo = vis_config.frame("book")
ob = vis_config.frame("book_obstacle")
to = vis_config.frame("stick")
vis_config.view()

for i, k, l, m in zip(path_list, object_list, obs_list, tool_list):
    bo.setPosition(k[0:3])
    bo.setQuaternion(k[3:])

    ob.setPosition(l[0:3])
    ob.setQuaternion(l[3:]) 

    to.setPosition([0.1, 0, 0.69])
    to.setQuaternion(m[3:])

    for j in i:
        print(j)
        vis_config.setJointState(j)
        
        vis_config.view()
        time.sleep(0.02)

In [ ]:
for i in config.getFrameNames():
    if "panda" in i or i in ["gripper", "palm", "finger1", "finger2", "box_ghost", "head_ghost"]:
        config.delFrame(i)

config.addFile("box_ghost.g")
config.frame("book_ghost").setPosition(object_list[20][0:3])
config.frame("book_ghost").setQuaternion(object_list[20][3:])

config.view()

In [ ]:
# # ### Visualization ###
environment="shelf_env.g"
robot="panda_tunnel.g"
vis_config = ry.Config()
vis_config.addFile(robot)
vis_config.addFile(environment)
vis_config.addFile("camera.g")
vis_config.view()
if np.linalg.norm(gen_wp - goal_pos) <threshold:
    print("TRAJECTORY FOUND!!!!")
    vis_config.view()
    time.sleep(2)
    for state in frame_list:  
        vis_config.setFrameState(state[0:87]) 
        vis_config.view()
        time.sleep(0.3)